<a href="https://colab.research.google.com/github/vaibhavmallya98/Smartknower-AI-Internship-Major-Project/blob/main/TranferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
directory_root = '/content/drive/My Drive/PlantVillage'

In [ ]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, (256,256))   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [ ]:
image_list, label_list = [], []
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)
    for directory in root_dir :
        # remove .DS_Store from list
        if directory == ".DS_Store" :
            root_dir.remove(directory)

    for plant_folder in root_dir :
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")
        
        for disease_folder in plant_disease_folder_list :
            # remove .DS_Store from list
            if disease_folder == ".DS_Store" :
                plant_disease_folder_list.remove(disease_folder)

        for plant_disease_folder in plant_disease_folder_list:
            print(f"[INFO] Processing {plant_disease_folder} ...")
            plant_disease_image_list = listdir(f"{directory_root}/{plant_folder}/{plant_disease_folder}/")
                
            for single_plant_disease_image in plant_disease_image_list :
                if single_plant_disease_image == ".DS_Store" :
                    plant_disease_image_list.remove(single_plant_disease_image)

            for image in plant_disease_image_list[:200]:
                image_directory = f"{directory_root}/{plant_folder}/{plant_disease_folder}/{image}"
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                    image_list.append(convert_image_to_array(image_directory))
                    label_list.append(plant_disease_folder)
    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error : {e}")

    

[INFO] Loading images ...
[INFO] Processing Tomato_Septoria_leaf_spot ...
[INFO] Processing Tomato_Leaf_Mold ...
[INFO] Processing Tomato_Bacterial_spot ...
[INFO] Processing Tomato_Spider_mites_Two_spotted_spider_mite ...
[INFO] Processing Tomato__Tomato_mosaic_virus ...
[INFO] Processing Tomato__Target_Spot ...
[INFO] Processing Tomato_Early_blight ...
[INFO] Processing Tomato__Tomato_YellowLeaf__Curl_Virus ...
[INFO] Processing Tomato_Late_blight ...
[INFO] Processing Tomato_healthy ...
[INFO] Processing Potato___Early_blight ...
[INFO] Processing Potato___healthy ...
[INFO] Processing Potato___Late_blight ...
[INFO] Processing Pepper__bell___healthy ...
[INFO] Processing Pepper__bell___Bacterial_spot ...
[INFO] Image loading completed


In [ ]:
print(len(image_list))
print(len(label_list))
print(type(image_list[0]))
print(image_list[0].shape)

2951
2951
<class 'numpy.ndarray'>
(256, 256, 3)


In [ ]:
#creating additional data by transposing every image with same lable
image_list_T = []
label_list_T = []
final_image_list = image_list
final_label_list = label_list
for i in range(len(image_list)):
  image_T = np.transpose(image_list[i],(1,0,2))
  image_list_T.append(image_T)
  label_list_T.append(label_list[i])

final_image_list+=image_list_T
final_label_list+=label_list_T

In [ ]:
len(final_image_list)

5902

In [ ]:
 from sklearn.preprocessing import LabelBinarizer
 encoder = LabelBinarizer()
 one_hot_label = encoder.fit_transform(label_list)
 print(one_hot_label)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


In [ ]:
unique_labels,indices = np.unique(label_list,return_index=True)
print(unique_labels)
print(indices)

['Pepper__bell___Bacterial_spot' 'Pepper__bell___healthy'
 'Potato___Early_blight' 'Potato___Late_blight' 'Potato___healthy'
 'Tomato_Bacterial_spot' 'Tomato_Early_blight' 'Tomato_Late_blight'
 'Tomato_Leaf_Mold' 'Tomato_Septoria_leaf_spot'
 'Tomato_Spider_mites_Two_spotted_spider_mite' 'Tomato__Target_Spot'
 'Tomato__Tomato_YellowLeaf__Curl_Virus' 'Tomato__Tomato_mosaic_virus'
 'Tomato_healthy']
[2751 2551 1999 2351 2199  400 1200 1599  200    0  600 1000 1400  800
 1799]


In [ ]:
label_indices = [np.argmax(one_hot_label[i]) for i in indices]
label_dict = {}
for i in range(len(label_indices)):
  label_dict[label_indices[i]]= unique_labels[i]
print(label_dict)   #will be used for evaluation

{0: 'Pepper__bell___Bacterial_spot', 1: 'Pepper__bell___healthy', 2: 'Potato___Early_blight', 3: 'Potato___Late_blight', 4: 'Potato___healthy', 5: 'Tomato_Bacterial_spot', 6: 'Tomato_Early_blight', 7: 'Tomato_Late_blight', 8: 'Tomato_Leaf_Mold', 9: 'Tomato_Septoria_leaf_spot', 10: 'Tomato_Spider_mites_Two_spotted_spider_mite', 11: 'Tomato__Target_Spot', 12: 'Tomato__Tomato_YellowLeaf__Curl_Virus', 13: 'Tomato__Tomato_mosaic_virus', 14: 'Tomato_healthy'}


In [ ]:
np_final_image_list = np.array(image_list) / 255.0

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(np_final_image_list, one_hot_label, test_size=0.2, random_state = 42)

In [ ]:
print(x_train.shape)
print(type(x_train))
print(len(x_train))

(4721, 256, 256, 3)
<class 'numpy.ndarray'>
4721


In [ ]:
import tensorflow as tf
from tensorflow.keras import models,layers
from keras.applications.inception_v3 import InceptionV3

In [ ]:
base_inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(256,256, 3))
base_inception.trainable = False

model = models.Sequential()
model.add(base_inception)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dropout(0.03))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(15, activation='softmax'))
model.summary()

87916544/87910968 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 6, 6, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_2 (Dense)              (None, 15)                7695      
Total params: 23,122,223
Trainable params: 1,319,439
Non-

In [ ]:
model.compile(loss="binary_crossentropy", optimizer='adam',metrics=["accuracy"])

In [ ]:
history = model.fit(x_train,y_train,epochs = 7)

Epoch 1/7
148/148 [==============================] - 612s 4s/step - loss: 0.1403 - accuracy: 0.5387
Epoch 2/7
148/148 [==============================] - 613s 4s/step - loss: 0.0834 - accuracy: 0.7437
Epoch 3/7
148/148 [==============================] - 613s 4s/step - loss: 0.0653 - accuracy: 0.8043
Epoch 4/7
148/148 [==============================] - 616s 4s/step - loss: 0.0507 - accuracy: 0.8430
Epoch 5/7
148/148 [==============================] - 617s 4s/step - loss: 0.0433 - accuracy: 0.8729
Epoch 6/7
148/148 [==============================] - 615s 4s/step - loss: 0.0380 - accuracy: 0.8854
Epoch 7/7
148/148 [==============================] - 614s 4s/step - loss: 0.0334 - accuracy: 0.9017


In [ ]:
test_prediction = model.predict(x_test)
test_prediction_classes= np.array([np.argmax(i) for i in test_prediction])
y_test_classes = np.array([np.argmax(i) for i in y_test])

In [ ]:
correct = 0
incorrect = 0 
for i in range(len(test_prediction_classes)):
  if y_test_classes[i] == test_prediction_classes[i]:
    correct+=1
  else:
    incorrect+=1
accuracy = correct/(correct+incorrect)
print(accuracy)
#Testing Accuracy

0.8052497883149873


In [ ]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test_classes, test_prediction_classes))
print(metrics.classification_report(y_test_classes, test_prediction_classes))

[[76  3  2  0  2  0  0  0  0  0  0  0  1  0  0]
 [ 5 62  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 75  2  0  0  1  0  0  0  0  0  0  0  0]
 [ 0  0  5 65  1  0  0  1  0  0  0  0  0  0  0]
 [ 0  0  0  6 54  0  1  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  0 71 11  1  0  7  0  0  1  1  1]
 [ 0  0  0  0  0  2 67  3  0  2  1  3  1  0  0]
 [ 0  0  2  4  0  1 10 57  1  0  1  3  1  0  0]
 [ 0  0  0  0  0  2 11  1 63  0  1  0  1  3  1]
 [ 0  0  1  0  0  5 16  0  1 50  0  1  1  3  0]
 [ 0  0  0  4  2  3  8  0  1  1 41 10  4  3  3]
 [ 2  0  0  2  0  2 13  1  1  0  4 56  1  0  5]
 [ 0  0  0  0  0  2  1  0  0  0  0  1 72  0  0]
 [ 0  0  0  0  0  0  5  0  0  0  0  0  1 76  0]
 [ 0  0  0  0  0  0  1  1  0  1  4  7  0  0 66]]
              precision    recall  f1-score   support

           0       0.92      0.90      0.91        84
           1       0.95      0.93      0.94        67
           2       0.88      0.96      0.92        78
           3       0.77      0.90      0.83        72
        

In [ ]:
train_prediction = model.predict(x_train)
train_prediction_classes= np.array([np.argmax(i) for i in train_prediction])
y_train_classes = np.array([np.argmax(i) for i in y_train])

In [ ]:
correct = 0
incorrect = 0 
for i in range(len(train_prediction_classes)):
  if y_train_classes[i] == train_prediction_classes[i]:
    correct+=1
  else:
    incorrect+=1
accuracy = correct/(correct+incorrect)
print(accuracy)
#Training Accuracy

0.9491633128574455


In [ ]:
from sklearn import metrics
print(metrics.confusion_matrix(y_train_classes, train_prediction_classes))
print(metrics.classification_report(y_train_classes, train_prediction_classes))

[[314   2   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  6 327   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0 322   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   4 321   2   0   1   0   0   0   0   0   0   0   0]
 [  0   0   0   7 236   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0 293   8   2   1   2   0   0   0   0   0]
 [  0   0   1   0   0   0 319   0   0   0   0   1   0   0   0]
 [  1   0   1   3   0   0  13 300   0   0   0   2   0   0   0]
 [  0   0   0   3   0   4  28   3 268   1   0   1   1   7   1]
 [  1   0   0   0   0   5  17   1   2 285   0   2   0   9   0]
 [  1   0   0   3   1   3  16   2   1   1 270  12   0   7   3]
 [  0   1   0   0   0   2  18   0   1   2   4 282   0   1   2]
 [  0   0   0   0   0   0   3   0   1   0   0   1 317   0   0]
 [  0   0   0   0   0   0   1   0   0   0   0   0   0 317   0]
 [  0   0   0   0   0   0   2   0   0   0   0   8   0   0 310]]
              precision    recall  f1-score   support


In [ ]:
model.save('PlantVillage_TransferLearning.h5')